In [40]:
def rank(G,c,i):
    """
    find occourances of charcter in string up to index i 
    args: 
    G input string
    c charicter were looking for
    i index to look up to 
    """
    j=0
    out=0
    while(j<i):
        if(G[j]==c):
            out=out+1
        j+=1
    return out 


def bwa_run(band_start, band_end, read, index, bwt_reference, bands):
    """
    runs the bwa algorithm on a single read. 
    args"
    band_start, band_end: (int) index postions of band in reference genome 
    read: (str)  a single sequence produced from a sequencer.
    index: (int) current position in read 
    """

    if index == -1:
        return band_start, band_end 
    if band_start == band_end:
        return -1
    s = read[index]
    rank_top = rank(bwt_reference,s,band_start)
    rank_bottom = rank(bwt_reference,s,band_end)
    return bwa_run(bands[read[index]]+rank_top, bands[read[index]]+rank_bottom, read , index-1, bands=bands,bwt_reference=bwt_reference)


def build_band_array(read, reference,v="$"):
    assert v not in reference,  "{0} already in input string".format(v)
    ex = reference+v
    bands={}
    backwards_read=read[::-1]
    bands[backwards_read[0]]=1
    for char in set(reference):
        if char not in backwards_read:
            backwards_read=backwards_read+char
    for i in range(1,len(backwards_read)):
        bands[backwards_read[i]]=bands[backwards_read[i-1]]+reference.count(backwards_read[i-1])
    bands[v]=0
    return bands


def suffix_array(reference, v='$'):
    """
    builds the suffix array 
    args:
    ref: (str) refereence genome
    v: (char) charicter not in refernce array to be added. defaults to $
    """
    
   # assert v not in reference,  "{0} already in input string".format(v)
    #reference = reference+v
    suffixes = [reference[i:] for i in range(len(reference))]
    suffix_array = sorted(range(len(reference)), key=lambda i: suffixes[i])
    bwt = ''.join([reference[i-1] for i in suffix_array])
    return bwt, suffix_array

def bwa(read, reference, v="$"):
    """
    whole bwa algorithm
    args:
    read: (str) indivudal reading
    ref: (str) refereence genome
    v: (char) charicter not in refernce array to be added. defaults to $
    """
    bands = build_band_array(read,reference)
    backwards_read = read[::-1]
    bwt_ref, suffix_ref=suffix_array(reference,v=v)
    print(bwt_ref)
    print(suffix_ref)
    indecies= bwa_run(bands[backwards_read[0]], bands[backwards_read[1]], read, index=len(read)-2, bwt_reference=bwt_ref , bands=bands)
    if indecies==-1:
        return "No match"
    print(indecies)
    print(len(suffix_ref))
    return suffix_ref[indecies[0]]
# read="GCAG"
# reference="CGATGCACCGGTACTGGATCGATCGATCGAGTGCTAGCGTAGCGAGGCATGGATCAG"
# print(len(reference))
# print(bwa(read, reference))


In [43]:
a=suffix_array(reference)
b=generate_suffix_array(reference)
a[1]==b

True

In [37]:
def generate_suffix_array(reference):
    return(list(sorted(range(len(reference)), key=lambda i:reference[i:])))

def bwt_from_suffix(string, s_array=None):
    if s_array is None:
        s_array = suffix_array(string)
    return("".join(string[idx - 1] for idx in s_array))


def lf_mapping(bwt, letters=None):
    if letters is None:
        letters = set(bwt)
        
    result = {letter:[0] for letter in letters}
    result[bwt[0]] = [1]
    for letter in bwt[1:]:
        for i, j in result.items():
            j.append(j[-1] + (i == letter))
    return(result)


from collections import Counter

def count_occurences(string, letters=None):
    count = 0
    result = {}
    
    if letters is None:
        letters = set(string)
        
    c = Counter(string)
    
    for letter in sorted(letters):
        result[letter] = count
        count += c[letter]
    return result


def update(begin, end, letter, lf_map, counts, string_length):
    beginning = counts[letter] + lf_map[letter][begin - 1] + 1
    ending = counts[letter] + lf_map[letter][end]
    return(beginning,ending)

def generate_all(input_string, s_array=None, eos="$"):
    letters = set(input_string)
    try:
        assert eos not in letters
    
        counts = count_occurences(input_string, letters)

        input_string = "".join([input_string, eos])
        if s_array is None:
            s_array = suffix_array(input_string)
        bwt = bwt_from_suffix(input_string, s_array)
        lf_map = lf_mapping(bwt, letters | set([eos]))

        for i, j in lf_map.items():
            j.extend([j[-1], 0])

        return letters, bwt, lf_map, counts, s_array

    except:
        print("End of string character found in text, deleted EOS from input string")
        input_string = input_string.replace(eos, "")
        letters = set(input_string)
        counts = count_occurences(input_string, letters)

        input_string = "".join([input_string, eos])
        if s_array is None:
            s_array = suffix_array(input_string)
        bwt = bwt_from_suffix(input_string, s_array)
        lf_map = lf_mapping(bwt, letters | set([eos]))

        for i, j in lf_map.items():
            j.extend([j[-1], 0]) 

        return letters, bwt, lf_map, counts, s_array

In [36]:
def find(search_string, input_string, mismatches=0, bwt_data=None, s_array=None):
    
    results = []
     
    if len(search_string) == 0:
        return("Empty Query String")
    if bwt_data is None:
        bwt_data = generate_all(input_string, s_array=s_array)
    
    letters, bwt, lf_map, count, s_array = bwt_data
    
    if len(letters) == 0:
        return("Empty Search String")

    if not set(search_string) <= letters:
        return []

    length = len(bwt)

    class Fuzzy(object):
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)

    fuz = [Fuzzy(search_string=search_string, begin=0, end=len(bwt) - 1,
                        mismatches=mismatches)]

    while len(fuz) > 0:
        p = fuz.pop()
        search_string = p.search_string[:-1]
        last = p.search_string[-1]
        all_letters = [last] if p.mismatches == 0 else letters
        for letter in all_letters:
            begin, end = update(p.begin, p.end, letter, lf_map, count, length)
            if begin <= end:
                if len(search_string) == 0:
                    results.extend(s_array[begin : end + 1])
                else:
                    miss = p.mismatches
                    if letter != last:
                        miss = max(0, p.mismatches - 1)
                    fuz.append(Fuzzy(search_string=search_string, begin=begin,
                                            end=end, mismatches=miss))
    return sorted(set(results))

In [18]:
read="GCAG"
reference="CGATGCACCGGTACTGGATCGATCGATCGAGTGCTAGCGTAGCGAGGCATGGATCAGGCAG"
find(read, reference, mismatches=0)

[57]